# 🗺️ MIT-Level Interactive Research Dashboard
## Multi-Agent Tour Guide System - Jupyter Integration

**Research Level:** MIT / Academic Publication  
**Version:** 1.0.0  
**Date:** November 2025

---

This notebook provides:
1. **Interactive Parameter Exploration** - Sliders and controls for real-time analysis
2. **Publication-Quality Visualizations** - Plotly-based interactive charts
3. **Statistical Analysis Tools** - Hypothesis testing and effect sizes
4. **Monte Carlo Simulations** - Stochastic system modeling
5. **Pareto Frontier Analysis** - Multi-objective optimization

### 🚀 Quick Start

For the full interactive dashboard experience, run:
```bash
python run_dashboard.py
```

This notebook provides a complementary Jupyter-based exploration.


In [1]:
# Core imports
import sys

sys.path.insert(0, "..")

import numpy as np
import pandas as pd
from scipy import stats
import warnings

warnings.filterwarnings("ignore")

# Plotly for interactive visualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set dark theme for Plotly
pio.templates.default = "plotly_dark"

# Dashboard components
from src.dashboard.data_manager import DashboardDataManager, QueueConfig
from src.dashboard.components import (
    SensitivityPanel,
    ParetoFrontierPanel,
    StatisticalComparisonPanel,
    MonteCarloPanel,
    AgentPerformancePanel,
    SystemMonitorPanel,
    COLORS,
)

# Initialize data manager
data_manager = DashboardDataManager()

print("✅ Dashboard components loaded successfully!")
print("\n📊 Available panels:")
print("   • SensitivityPanel")
print("   • ParetoFrontierPanel")
print("   • StatisticalComparisonPanel")
print("   • MonteCarloPanel")
print("   • AgentPerformancePanel")

✅ Dashboard components loaded successfully!

📊 Available panels:
   • SensitivityPanel
   • ParetoFrontierPanel
   • StatisticalComparisonPanel
   • MonteCarloPanel
   • AgentPerformancePanel


---

## 1. 🎲 Monte Carlo Simulation

Run a comprehensive Monte Carlo simulation with the default configuration.


In [2]:
# Configuration
N_SIMULATIONS = 10000
SOFT_TIMEOUT = 15.0  # seconds
HARD_TIMEOUT = 30.0  # seconds

print(f"🎲 Running Monte Carlo simulation with N={N_SIMULATIONS:,}...")
print(f"   Configuration: soft_timeout={SOFT_TIMEOUT}s, hard_timeout={HARD_TIMEOUT}s")

# Run simulation
config = QueueConfig(soft_timeout=SOFT_TIMEOUT, hard_timeout=HARD_TIMEOUT)
baseline_df = data_manager.get_baseline_simulation(
    N_SIMULATIONS, config, force_refresh=True
)

print(f"\n✅ Simulation complete!")
print(f"\n📊 Summary Statistics:")
print(f"   Mean Latency:   {baseline_df['latency'].mean():.3f}s")
print(f"   Std Latency:    {baseline_df['latency'].std():.3f}s")
print(f"   Mean Quality:   {baseline_df['quality'].mean():.3f}")
print(f"   Complete Rate:  {(baseline_df['status'] == 'complete').mean():.1%}")
print(f"   Success Rate:   {(baseline_df['status'] != 'failed').mean():.1%}")

🎲 Running Monte Carlo simulation with N=10,000...
   Configuration: soft_timeout=15.0s, hard_timeout=30.0s

✅ Simulation complete!

📊 Summary Statistics:
   Mean Latency:   5.648s
   Std Latency:    4.584s
   Mean Quality:   7.998
   Complete Rate:  85.6%
   Success Rate:   100.0%


In [3]:
# Create comprehensive simulation results visualization
fig = MonteCarloPanel.create_simulation_results(baseline_df)
fig.show()

---

## 2. 🔬 Interactive Sensitivity Analysis

Explore how different parameters affect system performance.


In [4]:
# Run sensitivity analysis for soft_timeout
print("🔬 Running sensitivity analysis for soft_timeout...")

soft_timeout_values = np.linspace(5, 30, 11).tolist()
sensitivity_df = data_manager.run_sensitivity_analysis(
    "soft_timeout", soft_timeout_values, n_sims=3000
)

# Create interactive sensitivity visualization
fig = SensitivityPanel.create_parameter_impact_chart(sensitivity_df, "soft_timeout")
fig.show()

🔬 Running sensitivity analysis for soft_timeout...


---

## 3. 🎯 Pareto Frontier Exploration

Explore the quality-latency tradeoff space to find optimal configurations.


In [5]:
# Generate Pareto frontier data
print("🎯 Generating Pareto frontier analysis...")
print("   This may take a minute...")

pareto_df = data_manager.run_pareto_analysis(
    soft_timeout_range=(5, 30), hard_timeout_range=(10, 60), n_points=49, n_sims=500
)

# 3D Surface and 2D Pareto visualization
fig = ParetoFrontierPanel.create_3d_pareto_surface(pareto_df)
fig.show()

fig2 = ParetoFrontierPanel.create_pareto_scatter(pareto_df)
fig2.show()

🎯 Generating Pareto frontier analysis...
   This may take a minute...


---

## 4. 📐 Statistical A/B Comparison

Compare two configurations with rigorous statistical testing.


In [6]:
# Define configurations to compare
config_a = QueueConfig(soft_timeout=15.0, hard_timeout=30.0)  # Default
config_b = QueueConfig(soft_timeout=8.0, hard_timeout=15.0)  # Aggressive

print("📐 Comparing configurations:")
print(
    f"   Config A: soft={config_a.soft_timeout}s, hard={config_a.hard_timeout}s (Default)"
)
print(
    f"   Config B: soft={config_b.soft_timeout}s, hard={config_b.hard_timeout}s (Aggressive)"
)

comparison = data_manager.compare_configurations(config_a, config_b, n_sims=10000)

# Distribution comparison visualization
fig = StatisticalComparisonPanel.create_distribution_comparison(
    comparison["df_a"], comparison["df_b"], ("Default (15s/30s)", "Aggressive (8s/15s)")
)
fig.show()

# Effect size visualization
fig2 = StatisticalComparisonPanel.create_effect_size_chart(comparison)
fig2.show()

# Print statistical summary
print("\n" + "=" * 70)
print("📊 STATISTICAL COMPARISON RESULTS")
print("=" * 70)
lat = comparison["latency"]
qual = comparison["quality"]
print(f"\n⏱️ LATENCY: p={lat['p_value']:.2e}, Cohen's d={lat['cohens_d']:.3f}")
print(f"⭐ QUALITY: p={qual['p_value']:.2e}, Cohen's d={qual['cohens_d']:.3f}")

📐 Comparing configurations:
   Config A: soft=15.0s, hard=30.0s (Default)
   Config B: soft=8.0s, hard=15.0s (Aggressive)



📊 STATISTICAL COMPARISON RESULTS

⏱️ LATENCY: p=8.38e-101, Cohen's d=0.304
⭐ QUALITY: p=8.97e-01, Cohen's d=-0.002


---

## 🎓 Summary & Launch Full Dashboard

For the complete interactive experience with real-time controls, sliders, and live updates:

```bash
# From the project root directory
python run_dashboard.py
```

Then open **http://localhost:8050** in your browser.

### Dashboard Features:
- 📊 **System Monitor** - Real-time agent health and throughput
- 🔬 **Sensitivity Analysis** - Interactive parameter exploration
- 🎯 **Pareto Explorer** - 3D quality-latency surfaces
- 📐 **A/B Testing** - Statistical comparison tools
- 🎲 **Monte Carlo** - Simulation controls

---
*MIT-Level Research Dashboard | Multi-Agent Tour Guide System | November 2025*
